# Analysing Statkraft and CERRA data

In [ ]:
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr

import datetime
import pandas as pd
from shapely.geometry import Point

import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS, Transformer
import seaborn as sns

from geopandas.tools import sjoin_nearest


import rioxarray

from mpl_toolkits.axes_grid1 import make_axes_locatable


STATKRAFT_CRS = "EPSG:32633"
REAL_CRS = "EPSG:32633"
ALTIMETRI_CRS = "EPSG:25832"
CERRA_CRS = "EPSG:4326"




In [ ]:
statkraft = pd.read_csv("data/processed_w_notebooks/statkraft_snow_depth_ordered.csv")
cerra = pd.read_csv("data/processed_w_notebooks/cerra_snow_depth_ordered.csv")
real = pd.read_csv("data/processed_w_notebooks/real_snow_depth_ordered.csv")
statkraft['time'] = pd.to_datetime(statkraft['time'])
cerra['time'] = pd.to_datetime(cerra['time'])
real['time'] = pd.to_datetime(real['time'])
real = real[real['time'] <= "2020-03-01"]
statkraft = statkraft.set_index("time")
cerra = cerra.set_index("time")
real = real.set_index("time")

In [ ]:
# statkraft = pd.read_csv("statkraft_snow_depth_ordered.csv")
# cerra = pd.read_csv("cerra_snow_depth_ordered.csv")
# real = pd.read_csv("real_snow_depth_ordered.csv")
# statkraft['time'] = pd.to_datetime(statkraft['time'])
# cerra['time'] = pd.to_datetime(cerra['time'])
# real['time'] = pd.to_datetime(real['time'])
# real = real[real['time'] <= "2020-03-01"]
# statkraft = statkraft.set_index("time")
# cerra = cerra.set_index("time")
# real = real.set_index("time")

In [ ]:
# Pivot the DataFrame to have units as columns
pivot_cerra = cerra.pivot_table(index='time', columns='unit', values='cerra_sde')
pivot_statkraft = statkraft.pivot_table(index='time', columns='unit', values='statkraft_sde')
pivot_real = real.pivot_table(index='time', columns='unit', values='real_sde')

In [ ]:
# Plot the time series for each unit with subplots
pivot_statkraft.plot(subplots=True, 
                     sharex=True,
                     #sharey=True, 
                     figsize=(10, 0.8 * len(pivot_statkraft.columns)), 
                     layout=(len(pivot_statkraft.columns), 1), 
                     legend=True)

# Set the x-axis label for the last subplot
plt.xlabel('Time')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Plot the time series for each unit with subplots
pivot_cerra.plot(subplots=True, 
                sharex=True, 
                sharey = True,
                figsize=(10, 0.8 * len(pivot_cerra.columns)), 
                layout=(len(pivot_cerra.columns), 1), 
                legend=True)

# Set the x-axis label for the last subplot
plt.xlabel('Time')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Plot the time series for each unit with subplots
fig, axes = plt.subplots(nrows=len(pivot_cerra.columns), 
                         ncols=1, 
                         linewidth=0.5,
                         sharex=True, 
                         figsize=(12, 2 * len(pivot_cerra.columns)))

for i, unit in enumerate(pivot_cerra.columns):
    ax = axes[i]
    pivot_cerra[unit].plot(ax=ax, label='CERRA', color='green')
    pivot_statkraft[unit].plot(ax=ax, label='Statkraft', color='orange')
    pivot_real[unit].plot(ax=ax, label='Real', color='blue')
    
    ax.set_title(f'Unit {unit}')
    ax.legend()

# Set the x-axis label for the last subplot
axes[-1].set_xlabel('Time')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Reset the index of each DataFrame to ensure unique index values
statkraft.reset_index(inplace=True)
cerra.reset_index(inplace=True)
real.reset_index(inplace=True)

# Concatenate the DataFrames along the columns
concat = pd.concat([statkraft, cerra, real], axis=1)

# Remove duplicate columns
concat = concat.loc[:, ~concat.columns.duplicated()]

# Display the concatenated DataFrame
print("Concatenated DataFrame:")
print(concat.head())

In [ ]:
# compute all errors
concat['err_statkraft'] = concat['statkraft_sde'] - concat['real_sde']
concat['err_cerra'] = concat['cerra_sde'] - concat['real_sde']
concat['err_statkraft_cerra'] = concat['statkraft_sde'] - concat['cerra_sde']
concat['abs_err_statkraft'] = concat['err_statkraft'].abs()
concat['abs_err_cerra'] = concat['err_cerra'].abs()
concat['abs_err_statkraft_cerra'] = concat['err_statkraft_cerra'].abs()

In [ ]:
plt.figure(figsize=(10, 6))  # set the figure size first
sns.boxplot(data=concat, x='unit', y='err_statkraft')
plt.title("Statkraft Error per Unit (vs. Real)")
plt.axhline(0, color='gray', linestyle='--')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with 3 subplots
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 15), sharex=True)

# Plot err_statkraft
sns.boxplot(data=concat, x='unit', y='err_statkraft', ax=axes[0])
axes[0].set_title("Statkraft Error per Unit (vs. Real)")
axes[0].axhline(0, color='gray', linestyle='--')
axes[0].set_xlabel('')
axes[0].set_ylabel('Error (m)')

# Plot err_cerra
sns.boxplot(data=concat, x='unit', y='err_cerra', ax=axes[1])
axes[1].set_title("CERRA Error per Unit (vs. Real)")
axes[1].axhline(0, color='gray', linestyle='--')
axes[1].set_xlabel('')
axes[1].set_ylabel('Error (m)')

# Plot err_statkraft_cerra
sns.boxplot(data=concat, x='unit', y='err_statkraft_cerra', ax=axes[2])
axes[2].set_title("Statkraft vs. CERRA Error per Unit")
axes[2].axhline(0, color='gray', linestyle='--')
axes[2].set_xlabel('Unit')
axes[2].set_ylabel('Error (m)')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
concat['month'] = concat['time'].dt.month
concat['season'] = concat['time'].dt.month % 12 // 3 + 1
concat['season'] = concat['season'].map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'})
concat['unit'] = concat['unit'].astype(int)

In [ ]:

plt.figure(figsize=(10, 6))  # set the figure size first
sns.violinplot(data=concat, x='season', y='err_statkraft');

#### “Does Statkraft consistently underestimate in Winter?”
#### “Is Spring the most uncertain season for the model?”

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set season order and pastel color palette
season_order = ['Winter', 'Spring', 'Summer', 'Autumn']
palette = sns.color_palette("pastel")

# Seaborn white background
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Ridgeline-style FacetGrid
g = sns.FacetGrid(
    concat,
    row="season",
    hue="season",
    aspect=6,
    height=2,  # This gives vertical space per row
    row_order=season_order,
    palette=palette
)

# 1. KDE fill
g.map(
    sns.kdeplot,
    "err_statkraft",
    fill=True,
    bw_adjust=0.6,
    clip_on=False,
    alpha=1,
    linewidth=1.5
)

# 2. Outline
g.map(
    sns.kdeplot,
    "err_statkraft",
    color="black",
    bw_adjust=0.6,
    linewidth=1,
    clip_on=False
)

# 3. Add season labels manually
def label(x, color, label):
    ax = plt.gca()
    ax.text(
        -0.05, 0.02, label,
        fontweight="bold",
        color=color,
        ha="left",
        va="center",
        transform=ax.transAxes
    )

g.map(label, "err_statkraft")

# Plot tweaks
g.figure.subplots_adjust(hspace=-0.45)  # tighter stacking
g.set_titles("")
g.set(yticks=[], ylabel="", xlabel="Statkraft Error (vs Real)")
g.despine(bottom=True, left=True)

plt.suptitle(
    "Statkraft Error Distribution per Season (Ridge Plot), Aggregated Over All Units",
    fontsize=16,
    ha='center',
    y=1.03
)

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure unit is string (for clearer labeling)
concat['unit'] = concat['unit'].astype(str)

# Create FacetGrid
g = sns.FacetGrid(
    concat,
    row="unit",
    hue="unit",
    aspect=10,
    height=1.2,
    palette="pastel"
)

# Filled KDE (pastel)
g.map(sns.kdeplot, "err_statkraft", fill=True, bw_adjust=0.5, alpha=0.8, linewidth=0)

# Black outline
g.map(sns.kdeplot, "err_statkraft", color="black", bw_adjust=0.5, linewidth=1)

# 🏷️ Add unit labels on left
def label(x, color, label):
    ax = plt.gca()
    ax.text(-0.15, 0, f"Unit {label}", fontweight="bold", color="black",
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "err_statkraft")

# Final layout cleanup
g.figure.subplots_adjust(hspace=-0.5)
g.set_titles("")
g.set(yticks=[], xlabel="Statkraft Error (vs Real)", ylabel="")
g.despine(bottom=True, left=True)

plt.suptitle("Statkraft Error Distribution per Unit", fontsize=14, ha='center')
plt.tight_layout()
plt.show()


In [ ]:
target_units = ['0', '4', '8', '9']

subset = concat[
    concat['unit'].isin(target_units) &
    concat['err_statkraft'].notna() &
    concat['season'].notna()
].copy()

print(f"Rows in subset: {len(subset)}")
print("Unique units:", subset['unit'].unique())
print("Seasons:", subset['season'].unique())

In [ ]:
target_units = ['0', '4', '8', '9']

subset = concat[
    concat['unit'].isin(target_units) &
    concat['err_statkraft'].notna() &
    concat['season'].notna()
].copy()

print(f"Rows in subset: {len(subset)}")
print("Unique units:", subset['unit'].unique())
print("Seasons:", subset['season'].unique())



# Ridge plot setup again
g = sns.FacetGrid(
    subset,
    row="season",
    col="unit",
    hue="season",
    aspect=1.2,
    height=1.5,
    sharex=True,
    palette="pastel",
    row_order=['Winter', 'Spring', 'Summer', 'Autumn']
)

g.map(sns.kdeplot, "err_statkraft", fill=True, alpha=0.8, linewidth=0, bw_adjust=0.6)
g.map(sns.kdeplot, "err_statkraft", color="black", linewidth=1, bw_adjust=0.6)

g.set_titles(col_template="Unit {col_name}", row_template="{row_name}")
g.set(yticks=[], xlabel="Statkraft Error", ylabel="")
g.despine(left=True)

plt.suptitle("Statkraft Error per Season {}".format(target_units), fontsize=14, ha='center')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
target_units = ['1', '2', '3', '5']

subset = concat[
    concat['unit'].isin(target_units) &
    concat['err_statkraft'].notna() &
    concat['season'].notna()
].copy()

print(f"Rows in subset: {len(subset)}")
print("Unique units:", subset['unit'].unique())
print("Seasons:", subset['season'].unique())



# Ridge plot setup again
g = sns.FacetGrid(
    subset,
    row="season",
    col="unit",
    hue="season",
    aspect=1.2,
    height=1.5,
    sharex=True,
    palette="pastel",
    row_order=['Winter', 'Spring', 'Summer', 'Autumn']
)

g.map(sns.kdeplot, "err_statkraft", fill=True, alpha=0.8, linewidth=0, bw_adjust=0.6)
g.map(sns.kdeplot, "err_statkraft", color="black", linewidth=1, bw_adjust=0.6)

g.set_titles(col_template="Unit {col_name}", row_template="{row_name}")
g.set(yticks=[], xlabel="Statkraft Error", ylabel="")
g.despine(left=True)

plt.suptitle("Statkraft Error per Season {}".format(target_units), fontsize=14, ha='center')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
target_units = ['6', '7', '10', '11']

subset = concat[
    concat['unit'].isin(target_units) &
    concat['err_statkraft'].notna() &
    concat['season'].notna()
].copy()

print(f"Rows in subset: {len(subset)}")
print("Unique units:", subset['unit'].unique())
print("Seasons:", subset['season'].unique())



# Ridge plot setup again
g = sns.FacetGrid(
    subset,
    row="season",
    col="unit",
    hue="season",
    aspect=1.2,
    height=1.5,
    sharex=True,
    palette="pastel",
    row_order=['Winter', 'Spring', 'Summer', 'Autumn']
)

g.map(sns.kdeplot, "err_statkraft", fill=True, alpha=0.8, linewidth=0, bw_adjust=0.6)
g.map(sns.kdeplot, "err_statkraft", color="black", linewidth=1, bw_adjust=0.6)

g.set_titles(col_template="Unit {col_name}", row_template="{row_name}")
g.set(yticks=[], xlabel="Statkraft Error", ylabel="")
g.despine(left=True)

plt.suptitle("Statkraft Error per Season {}".format(target_units), fontsize=14, ha='center')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
target_units = ['12', '13', '14', '15']

subset = concat[
    concat['unit'].isin(target_units) &
    concat['err_statkraft'].notna() &
    concat['season'].notna()
].copy()

print(f"Rows in subset: {len(subset)}")
print("Unique units:", subset['unit'].unique())
print("Seasons:", subset['season'].unique())



# Ridge plot setup again
g = sns.FacetGrid(
    subset,
    row="season",
    col="unit",
    hue="season",
    aspect=1.2,
    height=1.5,
    sharex=True,
    palette="pastel",
    row_order=['Winter', 'Spring', 'Summer', 'Autumn']
)

g.map(sns.kdeplot, "err_statkraft", fill=True, alpha=0.8, linewidth=0, bw_adjust=0.6)
g.map(sns.kdeplot, "err_statkraft", color="black", linewidth=1, bw_adjust=0.6)

g.set_titles(col_template="Unit {col_name}", row_template="{row_name}")
g.set(yticks=[], xlabel="Statkraft Error", ylabel="")
g.despine(left=True)

plt.suptitle("Statkraft Error per Season {}".format(target_units), fontsize=14, ha='center')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# Load shapefile
catchment = gpd.read_file("data/raw/catchment_statkraft/catchment.shp")
catchment = catchment.to_crs("EPSG:4326")  # Ensure CRS match


statkraft_unit_errors = concat.groupby('unit').agg({
    'err_statkraft': 'mean',
    'abs_err_statkraft': 'mean',
    'latitude': 'first',
    'longitude': 'first',
}).reset_index()

statkraft_cerra_unit_errors = concat.groupby('unit').agg({
    'err_statkraft_cerra': 'mean',
    'abs_err_statkraft_cerra': 'mean',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()

statkraft_unit_seasonal_errors = concat.groupby(['unit', 'season']).agg({
    'err_statkraft': 'mean',
    'abs_err_statkraft': 'mean',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()

statkraft_gdf = gpd.GeoDataFrame(
    statkraft_unit_errors,
    geometry=gpd.points_from_xy(statkraft_unit_errors['longitude'], 
                                statkraft_unit_errors['latitude']),
    crs="EPSG:4326"
)

statkraft_cerra_gdf = gpd.GeoDataFrame(
    statkraft_cerra_unit_errors,
    geometry=gpd.points_from_xy(statkraft_cerra_unit_errors['longitude'], 
                                statkraft_cerra_unit_errors['latitude']),
    crs="EPSG:4326"
)

statkraft_seasonal_gdf = gpd.GeoDataFrame(
    statkraft_unit_seasonal_errors,
    geometry=gpd.points_from_xy(statkraft_unit_seasonal_errors.longitude, statkraft_unit_seasonal_errors.latitude),
    crs="EPSG:4326"
)

In [ ]:
#Plot error points: diverging colormap centered at 0

catchment_plot = catchment.to_crs("EPSG:4326")


gdf = statkraft_gdf
column_name = 'err_statkraft'


# Plot setup
fig, ax = plt.subplots(figsize=(8, 6))


vmin = gdf[column_name].min()
vmax = gdf[column_name].max()
abs_max = max(abs(vmin), abs(vmax))  # ensures symmetrical color scale

# Plot error points
gdf.plot(
    column=column_name,
    cmap='RdBu_r',  # diverging: blue = negative, red = positive
    legend=True,
    vmin=-abs_max,
    vmax=abs_max,
    ax=ax,
    markersize=80,
    edgecolor='black',
    linewidth=0.5
)

# Plot catchment boundary
catchment.boundary.plot(ax=ax, edgecolor="black", linewidth=2)

# Add unit labels on top of each point
for idx, row in gdf.iterrows():
    unit = row['unit'] if 'unit' in gdf.columns else idx  # fallback to index if no unit column
    x, y = row.geometry.x, row.geometry.y
    ax.text(x, y + 0.01, str(unit), fontsize=9, ha='center', va='bottom', color='black')

# Title and legend
ax.set_title("Mean {} by Location".format(column_name))



plt.tight_layout()
plt.show();


In [ ]:
# Setup
column_name = 'err_statkraft'
seasons = ['Winter', 'Spring', 'Summer', 'Autumn']
season_order = ['Winter', 'Spring', 'Summer', 'Autumn']

# Shared color scale
vmin = statkraft_gdf[column_name].min()
vmax = statkraft_gdf[column_name].max()
abs_max = max(abs(vmin), abs(vmax))

# Setup figure
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
axs = axs.flatten()

for i, season in enumerate(season_order):
    ax = axs[i]

    # Filter GeoDataFrame by season
    gdf_season = statkraft_seasonal_gdf[statkraft_seasonal_gdf['season'] == season]

    # Plot error points
    gdf_season.plot(
        column=column_name,
        cmap='RdBu_r',
        legend=False,  # one colorbar later
        vmin=-abs_max,
        vmax=abs_max,
        ax=ax,
        markersize=80,
        edgecolor='black',
        linewidth=0.5
    )

    # Catchment boundary
    catchment.boundary.plot(ax=ax, edgecolor="black", linewidth=2)

    # Unit labels
    for idx, row in gdf_season.iterrows():
        unit = row['unit']
        x, y = row.geometry.x, row.geometry.y
        ax.text(x, y + 0.01, str(unit), fontsize=9, ha='center', va='bottom', color='black')

    ax.set_title(f"{season}")
    ax.axis('off')

# Colorbar (shared)
sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=plt.Normalize(vmin=-abs_max, vmax=abs_max))
sm._A = []
cbar = fig.colorbar(sm, ax=axs, orientation='horizontal', fraction=0.03, pad=0.07)
cbar.set_label("Statkraft Error (Statkraft - CERRA)")

fig.suptitle("Statkraft Mean Error by Season and Unit Location", fontsize=16)
#plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# Filter only Winter and Spring
filtered = concat[concat['season'].isin(['Winter', 'Spring'])].copy()

# Group by unit and month
monthly_errors = filtered.groupby(['unit', 'month']).agg({
    'err_statkraft': 'mean',
    'abs_err_statkraft': 'mean',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()

# Convert to GeoDataFrame

monthly_gdf = gpd.GeoDataFrame(
    monthly_errors,
    geometry=gpd.points_from_xy(monthly_errors.longitude, monthly_errors.latitude),
    crs="EPSG:4326"
)


In [ ]:
months_to_plot = range(1,13)
month_labels = {
    1: "Jan", 2: "Feb", 3: "Mar", 
    4: "Apr", 5: "May", 6: "Jun",
    7: "Jul", 8: "Aug", 9: "Sep",
    10: "Oct", 11: "Nov", 12: "Dec",
}

# Setup figure

for i, month in enumerate(months_to_plot):
    gdf_month = monthly_gdf[monthly_gdf['month'] == month]
    print(f"Month {month}: {len(gdf_month)} points")


In [ ]:
updated_months_to_plot = [1,2,3,4,5,12]
month_labels = {
    1: "Jan", 2: "Feb", 3: "Mar", 
    4: "Apr", 5: "May", 6: "Jun",
    7: "Jul", 8: "Aug", 9: "Sep",
    10: "Oct", 11: "Nov", 12: "Dec",
}

# Shared color scale across all months
vmin = monthly_gdf['err_statkraft'].min()
vmax = monthly_gdf['err_statkraft'].max()
abs_max = max(abs(vmin), abs(vmax))


fig, axs = plt.subplots(2, 3, figsize=(18, 14))
axs = axs.flatten()

for i, month in enumerate(updated_months_to_plot):
    ax = axs[i]

    gdf_month = monthly_gdf[monthly_gdf['month'] == month]

    if gdf_month.empty:
        ax.set_title(f"{month_labels[month]} (no data)")
        ax.axis('off')
        continue

    # Plot error points
    gdf_month.plot(
        column='err_statkraft',
        cmap='RdBu_r',
        legend=False,
        vmin=-abs_max,
        vmax=abs_max,
        ax=ax,
        markersize=80,
        edgecolor='black',
        linewidth=0.5
    )

    # Catchment overlay
    catchment.boundary.plot(ax=ax, edgecolor="black", linewidth=2)

    # Unit labels
    for idx, row in gdf_month.iterrows():
        ax.text(row.geometry.x, row.geometry.y + 0.01, str(row['unit']),
                fontsize=9, ha='center', va='bottom', color='black')

    ax.set_title(f"Statkraft Error - {month_labels[month]}")
    ax.axis('off')

sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=mpl.colors.Normalize(vmin=-abs_max, vmax=abs_max))
sm._A = []
cbar = fig.colorbar(sm, ax=axs, orientation='horizontal', fraction=0.03, pad=0.07)
cbar.set_label("Statkraft Error (Statkraft - CERRA)")

plt.suptitle("Monthly Statkraft Model Error Across All Units", fontsize=16)
#plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show();


In [ ]:
concat.columns

In [ ]:
# Create a distribution plot for each unit
g = sns.FacetGrid(concat, col="unit", col_wrap=4, height=4, sharex=False, sharey=False)
g.map(sns.histplot, 'err_statkraft', bins=30, kde=True)

# Set axis labels and titles
g.set_axis_labels("Difference (m)", "Frequency")
g.set_titles("Unit {col_name}")

# Set the overall title
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Distribution of Statkraft vs CERRA error per unit')

# Show the plot
plt.show()

In [ ]:
err_long = concat.melt(
    id_vars=['time', 'unit'],
    value_vars=['err_statkraft', 'err_cerra', 'err_statkraft_cerra'],
    var_name='error_type',
    value_name='error'
)

In [ ]:
plt.figure(figsize=(8, 4))
sns.violinplot(data=err_long, hue='error_type', y='error', palette='Set2')

plt.axhline(0, color='gray', linestyle='--')
plt.title("Distribution of Errors Across Models")
plt.ylabel("Error (m)")

# Fix the legend background
legend = plt.legend(title='error_type')
legend.get_frame().set_facecolor('white')  # make legend box white
legend.get_frame().set_edgecolor('black')  # optional: add border
legend.get_frame().set_alpha(1.0)          # optional: remove transparency

plt.show()

From the plot we notice that the Statkraft model is underestimating sde, and we get some positive outliers for overestimating

In [ ]:
unit_subset = concat[concat['unit'] == '0']
unit_subset.set_index('time')[['err_statkraft', 'err_cerra', 'err_statkraft_cerra']].plot(figsize=(12, 3))
plt.axhline(0, color='gray', linestyle='--')
plt.title("Error Comparison Over Time (Unit 0)")
plt.ylabel("Error (m)")
plt.show()

In [ ]:
concat['which_is_closer'] = np.where(
    concat['abs_err_statkraft'] < concat['abs_err_cerra'],
    'statkraft', 'cerra'
)

sns.countplot(data=concat, x='which_is_closer')
plt.title("Which Model is Closer to Real (by Time Point)")

## Does high correlation between units mean similar statkraft errors?

In [ ]:
# Correlation matrix between units
corr_matrix = pivot_real.corr()
statkraft_corr = concat.pivot(index='time', columns='unit', values='statkraft_sde').corr()
cerra_corr = concat.pivot(index='time', columns='unit', values='cerra_sde').corr()
# Drop self-correlation (diagonal = 1)
mean_corr = corr_matrix.apply(lambda row: row.drop(row.name).mean(), axis=1)
#average model error per unit
unit_errors = concat.groupby('unit')['err_statkraft'].mean()

In [ ]:

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, 
            #annot=True, 
            fmt=".2f", cmap="coolwarm", square=True, linewidths=0.5, cbar_kws={'label': 'Correlation'})
plt.title("Correlation Matrix of Units Based on real_sde")
plt.xlabel("Unit")
plt.ylabel("Unit")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(statkraft_corr, 
            #annot=True, 
            fmt=".2f", cmap="coolwarm", square=True, linewidths=0.5, cbar_kws={'label': 'Correlation'})
plt.title("Correlation Matrix of Units Based on statkraft_sde")
plt.xlabel("Unit")
plt.ylabel("Unit")
plt.tight_layout()
plt.show()

Statkraft capture the correlations between the units that appear in the real snow depth data

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cerra_corr, 
            #annot=True, 
            fmt=".2f", cmap="coolwarm", square=True, linewidths=0.5, cbar_kws={'label': 'Correlation'})
plt.title("Correlation Matrix of Units Based on cerra_sde")
plt.xlabel("Unit")
plt.ylabel("Unit")
plt.tight_layout()
plt.show()

### Cluster correlation maps

In [ ]:
g = sns.clustermap(corr_matrix, cmap="coolwarm", figsize=(10, 10), annot=False)
g.fig.suptitle("Correlation for real_sde Cluster Map", fontsize=16, y=1.02)  
plt.show()

In [ ]:
g = sns.clustermap(statkraft_corr, cmap="coolwarm", figsize=(10, 10), annot=False)
g.fig.suptitle("Correlation for statkraft_sde Cluster Map", fontsize=16, y=1.02)  
plt.show()

In [ ]:
g = sns.clustermap(cerra_corr, cmap="coolwarm", figsize=(10, 10), annot=False)
g.fig.suptitle("Correlation for cerra_sde Cluster Map", fontsize=16, y=1.02) 
plt.show()

#### Is there any site that stands out and has a different behaviour from the rest?

In [ ]:

# Add to gdf
gdf['unit'] = gdf['unit'].astype(int)
gdf['mean_corr_real'] = gdf['unit'].map(mean_corr)

fig, ax = plt.subplots(figsize=(8, 6))

# Plot correlation as color
gdf.plot(column='mean_corr_real', cmap='viridis', legend=True, ax=ax)

# Catchment boundary
catchment.boundary.plot(ax=ax, edgecolor="black", linewidth=2)

# Add unit labels
for idx, row in gdf.iterrows():
    unit = int(row['unit'])
    x, y = row.geometry.x, row.geometry.y
    ax.text(x, y + 0.01, str(unit), fontsize=9, ha='center', va='bottom', color='black')

# Title and layout
ax.set_title("Mean Correlation of Unit with Others (based on real_sde)")
plt.tight_layout()
plt.show()


### Exploring Terrain and checking whether it is correlated with statkraft_error

In [ ]:
dem = rioxarray.open_rasterio("data/processed_w_notebooks/catchment_terrain_dem.tif", masked=True)

In [ ]:
# Reproject catchment and gdf (units) to DEM CRS
catchment_proj = catchment.to_crs(dem.rio.crs)
gdf_proj = gdf.to_crs(dem.rio.crs)

# Plot
fig, ax = plt.subplots(figsize=(7,5))

# 1. Terrain background
dem.squeeze().plot(
    ax=ax,
    cmap='terrain',
    cbar_kwargs={'label': 'Elevation (m)'},
    alpha=0.9
)

# 2. Catchment boundary
catchment_proj.boundary.plot(ax=ax, edgecolor="black", linewidth=2)

# 3. Unit points (no coloring, just black points)
gdf_proj.plot(ax=ax, 
              color='red', 
              markersize=60,
            edgecolor='white', 
            linewidth=0.5)

# 4. Labels
for idx, row in gdf_proj.iterrows():
    x, y = row.geometry.x, row.geometry.y
    ax.text(x, y + 200, str(row['unit']), fontsize=9, ha='center', va='bottom', color='black')

# Formatting
ax.set_title("Catchment Area with Unit Locations over Terrain", fontsize=14)
ax.set_axis_off()
plt.tight_layout()
plt.show()


In [ ]:
catchment_proj = catchment.to_crs(dem.rio.crs)
gdf_proj = statkraft_gdf.to_crs(dem.rio.crs)  # ← your GeoDataFrame with errors

column_name = 'err_statkraft'

# Symmetric color range around 0
vmin = gdf_proj[column_name].min()
vmax = gdf_proj[column_name].max()
abs_max = max(abs(vmin), abs(vmax))

# Plot
fig, ax = plt.subplots(figsize=(8,6))

# 🔹 1. Background terrain
dem.squeeze().plot(
    ax=ax,
    cmap='terrain',
    alpha=0.85,
    cbar_kwargs={
        'label': 'Elevation (m)',
        'shrink': 0.5,
        'fraction': 0.03,
        'pad': 0.02,
        'aspect': 20
    }
)

# 🔹 2. Catchment boundary
catchment_proj.boundary.plot(ax=ax, edgecolor="black", linewidth=2)

from mpl_toolkits.axes_grid1 import make_axes_locatable

# Create a divider for the existing axis
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="5%", pad=0.4)

gdf_proj.plot(
    column=column_name,
    cmap='RdBu_r',
    vmin=-abs_max,
    vmax=abs_max,
    ax=ax,
    markersize=80,
    edgecolor='black',
    linewidth=0.5,
    legend=True,
    legend_kwds={
        'orientation': 'horizontal',
        'cax': cax,
        'label': 'Error'
    }
)


# 🔹 4. Unit labels
for idx, row in gdf_proj.iterrows():
    x, y = row.geometry.x, row.geometry.y
    ax.text(x, y + 200, str(row['unit']), fontsize=9, ha='center', va='bottom', color='black')

# Final formatting
ax.set_title("Statkraft Error by Unit Over Terrain", fontsize=14)
ax.set_axis_off()
plt.tight_layout()
plt.show()


In [ ]:
#elevation_df_with_latlon = pd.read_csv("data/processed_w_notebooks/catchment_elevation.csv")
elevation_df_with_latlon = pd.read_csv("data/processed_w_notebooks/catchment_elevation.csv")

In [ ]:
units_gdf = gpd.GeoDataFrame(
    statkraft_unit_errors,
    geometry=gpd.points_from_xy(statkraft_unit_errors['longitude'], statkraft_unit_errors['latitude']),
    crs="EPSG:4326"
)

elev_gdf = gpd.GeoDataFrame(
    elevation_df_with_latlon,
    geometry=gpd.points_from_xy(elevation_df_with_latlon['longitude'], elevation_df_with_latlon['latitude']),
    crs="EPSG:4326"
)

units_proj = units_gdf.to_crs("EPSG:25832")
elev_proj = elev_gdf.to_crs("EPSG:25832")

# This assigns each DEM pixel to the nearest unit point
elev_with_unit = sjoin_nearest(
    elev_proj,
    units_proj[['unit', 'geometry']],
    how="left"
)

mean_elevation_per_unit = elev_with_unit.groupby('unit')['elevation'].mean().reset_index()
statkraft_with_elevation = statkraft_unit_errors.merge(mean_elevation_per_unit, on='unit', how='left')


In [ ]:
mean_elevation_per_unit = elev_with_unit.groupby('unit')['elevation'].mean().reset_index()
statkraft_with_elevation = statkraft_unit_errors.merge(mean_elevation_per_unit, on='unit', how='left')


In [ ]:
# Create elevation bins
statkraft_with_elevation['elevation_bin'] = pd.cut(
    statkraft_with_elevation['elevation'],
    bins=5
)

# Boxplot of error in each elevation range
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=statkraft_with_elevation,
    hue='elevation_bin',
    y='err_statkraft',
    palette='coolwarm'
)
plt.xticks(rotation=45)
plt.xlabel("Elevation Bins (m)")
plt.ylabel("Statkraft Error")
plt.title("Distribution of Statkraft Error Across Elevation Ranges")
plt.grid(True)
plt.tight_layout()

# Fix the legend background

legend.get_frame().set_facecolor('white')  # make legend box white
legend.get_frame().set_edgecolor('black')  # optional: add border
legend.get_frame().set_alpha(1.0)          # optional: remove transparency

plt.show()


In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 7))

# Scatter plot with larger, semi-transparent markers
scatter = plt.scatter(
    statkraft_with_elevation['elevation'],
    statkraft_with_elevation['err_statkraft'],
    c=statkraft_with_elevation['abs_err_statkraft'],
    cmap='viridis',
    s=100,
    edgecolor='k',
    alpha=0.7
)
plt.colorbar(scatter, label='Absolute Error')

# Trend line (optional, still helpful)
sns.regplot(
    x='elevation', y='err_statkraft',
    data=statkraft_with_elevation,
    scatter=False,
    color='red',
    line_kws={'linewidth': 2, 'label': 'Trend Line'}
)

# Add labels to the *right* of each marker
for _, row in statkraft_with_elevation.iterrows():
    plt.annotate(
        row['unit'],
        (row['elevation'], row['err_statkraft']),
        textcoords="offset points",
        xytext=(6, 0),  # (x, y) offset in points
        ha='left',
        fontsize=9,
        weight='bold'
    )

plt.xlabel("Mean Elevation per Unit (m)", fontsize=12)
plt.ylabel("Mean Statkraft Error per Unit", fontsize=12)
plt.title("Elevation vs Statkraft Error per Unit", fontsize=14)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
